# 1. Given billions of numbers (assume each 
record has a single number), find minimum, 
maximum, average for all numbers. 

In [1019]:
we=[2,4,5,0,9,90,80]

In [1020]:
rdd=spark.sparkContext.parallelize(we)

In [1021]:
def find_min_max(Partitions):
    first_time=False
    min2=0
    max2=0
    for val in Partitions:
        if first_time==False:
            min2=val
            max2=val
            first_time=True
        else:
            min2=min(min2,val)
            max2=max(max2,val)
    
    return [(min2, max2)]

In [1022]:
rdd200=rdd.mapPartitions(find_min_max)

In [1023]:
rdd201=rdd.reduce(lambda x,y: x+y )

In [1008]:
rdd202=rdd.count()

In [1009]:
rdd203=rdd201/rdd202

In [1010]:
rdd203

27.142857142857142

In [1027]:
min_max=rdd200.reduce(lambda x, y: (min(x[0], y[0]), max(x[1], y[1]), rdd203))

In [1028]:
min_max

(0, 90, 27.142857142857142)

=========================================

# 2. Given billions of numbers (assume each 
record has a single number), find count of
zeros, positives, and negatives

In [92]:
nums=[2,4,5,0,9,90,80,0,-9,-2]

In [93]:
def counting(Partitions):
    ze=0
    nega=0
    posi=0
    for val in Partitions: 
        if val<0:
            nega+=1
        elif (val ==0):
            ze+=1
        else:
            posi+=1
    return [(ze, posi, nega)]
        

In [94]:
rdd2=spark.sparkContext.parallelize(nums)

In [95]:
rdd3=rdd2.mapPartitions(counting)

In [96]:
rdd3.collect()

[(0, 2, 0), (1, 1, 0), (0, 2, 0), (1, 1, 2)]

In [97]:
rdd4= rdd3.reduce(lambda x,y: ((x[0]+y[0]), (x[1]+y[1]), (x[2]+y[2]) ))

In [98]:
rdd4

(2, 6, 2)

=======================================================================

# 8. Consider the following input record format:
<studentID , single-grade-in-range-of-0-to-100>

The goal is to find minimum and maximum grades 
for all students. Write a PySpark program to 
accomplish this task. Your output will be 

<studentID, minimum-grade, maximum-grade>

In [210]:
inp=(1, 10), (2,80), (3,55), (1,75),(2, 90), (1, 20), (3, 50), 

In [211]:
rdd5=spark.sparkContext.parallelize(inp)

In [212]:
rdd5.collect()

[(1, 10), (2, 80), (3, 55), (1, 75), (2, 90), (1, 20), (3, 50)]

In [219]:
rdd8=rdd5.groupByKey()

In [224]:
rdd9=rdd8.mapValues(lambda x: (min(x), max(x)))

In [225]:
rdd9.collect()

[(1, (10, 75)), (2, (80, 90)), (3, (50, 55))]

=======================================================================

# 9 Consider the following input record format:
<movie-name, rating-in-range-of-1-to-5>

The goal is to find the number of raters per movie
Write a PySpark program to accomplish this task. 
Your output will be 
<movie-name, number-of-raters>



In [227]:
movies=('a', 4), ('b', 4), ('c', 3), ('a',1),('a', 2),('b', 2),('a',1),('a', 2)

In [228]:
rdd10=spark.sparkContext.parallelize(movies)

In [229]:
rdd11=rdd10.groupByKey()

In [230]:
ans=rdd11.mapValues(lambda x: len(x))

In [232]:
ans.collect()

[('b', 2), ('c', 1), ('a', 5)]

# 10. Assume the following input
<gene-ID , reference , gene-value>

where reference can be:
"r1": as normal
"r2": as cancer
"r3": as unknown

The goal is to write a PySpark program to 
keep only normal genes and finally count 
them for all genes.

In [234]:
genes = ("g1", "r1", 3), ("g2", "r1", 4), ("g3", "r2", 3), ("g4", "r2", 1), ("g5", "r3", 3)

In [235]:
rdd12=spark.sparkContext.parallelize(genes)

In [236]:
rdd13=rdd12.filter(lambda x: x[1]=="r1")

In [237]:
rdd13.collect()

[('g1', 'r1', 3), ('g2', 'r1', 4)]

In [238]:
rdd13.count()

2

=======================================================================

# 11. Let a bigram be defined as a sequence 
of two consecutive words. For example for 
the following input: "w1,w2,w3,w4",
we can construct the following bigrams:

w1, w2
w2, w3
w3, w4

Let your input be a text file (x.dat), where 
each record has the following format (a record 
may have any number of words):
<word1,word2,word3>...

Write a PySpark program to find frequency 
of all unique bigrams.


In [1029]:
input_path= '/Users/supriyatiwari/documents/PySpark_docs/bigram.txt'

In [1030]:
rdd14= spark.sparkContext.textFile(input_path)

In [1031]:
rdd14.collect()

['words me you her she is mad but me we', 'mad but me we', 'words me you']

In [1034]:
rdd15=rdd14.map(lambda x: x.split(" "))

In [1035]:
rdd15.collect()

[['words', 'me', 'you', 'her', 'she', 'is', 'mad', 'but', 'me', 'we'],
 ['mad', 'but', 'me', 'we'],
 ['words', 'me', 'you']]

In [1042]:
rdd16=rdd15.flatMap(lambda x: [((x[i], x[i+1]), 1) for i in range (0,len(x)-1)])

In [1043]:
rdd16.collect()

[(('words', 'me'), 1),
 (('me', 'you'), 1),
 (('you', 'her'), 1),
 (('her', 'she'), 1),
 (('she', 'is'), 1),
 (('is', 'mad'), 1),
 (('mad', 'but'), 1),
 (('but', 'me'), 1),
 (('me', 'we'), 1),
 (('mad', 'but'), 1),
 (('but', 'me'), 1),
 (('me', 'we'), 1),
 (('words', 'me'), 1),
 (('me', 'you'), 1)]

In [1046]:
rdd17 = rdd16.reduceByKey(lambda x,y: (x+y))

In [1047]:
rdd17.collect()

[(('words', 'me'), 2),
 (('me', 'you'), 2),
 (('you', 'her'), 1),
 (('her', 'she'), 1),
 (('she', 'is'), 1),
 (('is', 'mad'), 1),
 (('mad', 'but'), 2),
 (('but', 'me'), 2),
 (('me', 'we'), 2)]

==========================================================================================

# 15. Provide solution in MapReduce and PySpark
Given the following input, write a classic 
map() and reduce() functions to find the maximum 
and minimum of all given keys and values for all 
given records.  For the following input (listed 
below), the output (output of all reducers) will 
be:

min  10
max  90

Input is given as a set of (K, V) pairs:

90    20
40    70
10    40
30    40
40    90
30    80
20    30
20    10

In [54]:
nums = [(90, 20), (40, 70), (10,40), (30,40)]

In [55]:
rdd1=spark.sparkContext.parallelize(nums)

In [56]:
rdd1.count()

4

In [57]:
rdd_new = rdd1.flatMap(lambda x:x)

In [58]:
rdd_new.count()

8

In [61]:
rdd_new.collect()

[90, 20, 40, 70, 10, 40, 30, 40]

In [64]:
rdd_min=rdd_new.reduce(lambda x, y: min(x, y))

In [67]:
rdd_max=rdd_new.reduce(lambda x, y: max(x, y))

In [70]:
print("min:" + str(rdd_min) + "," + "max:" + str(rdd_max))

min:10,max:90


==========================================================================================

# 16. Given the following rdd in pyspark:

data = ['k1', 'k2', 'k1', 'k2', 
            'k1', 'k2', 'k3', 'k2']
rdd = sc.parallelize(data)

write a sequence of pyspark transformations and actions to 
find frequencies  of all keys in data. For this example, your 
solution should generate/output:
('k1', 3)
('k2', 4)

If a frequency is less than 2, then drop them.

In [275]:
data_n = ['k1', 'k2', 'k1', 'k2', 'k1', 'k2', 'k3', 'k2'] 

In [276]:
rdd18 = sc.parallelize(data_n)

In [277]:
rdd19=rdd18.map(lambda x: (x,1))

In [278]:
rdd19.collect()

[('k1', 1),
 ('k2', 1),
 ('k1', 1),
 ('k2', 1),
 ('k1', 1),
 ('k2', 1),
 ('k3', 1),
 ('k2', 1)]

In [279]:
rdd20=rdd19.reduceByKey(lambda x,y: (x+y))

In [280]:
rdd20.collect()

[('k1', 3), ('k2', 4), ('k3', 1)]

In [281]:
final_r=rdd20.filter(lambda x: x[1]>1)

In [282]:
final_r.collect()

[('k1', 3), ('k2', 4)]

==========================================================================================

# 19. PySpark and MapReduce solutions.
Given large set of documents, we want to use 
classic MapReduce to create an "inverted index" 
for all documents. For example, given the following 
input documents:

Document1: fox jumped fast fox fast
Document2: fox ran fox jumped fast
Document3: hello hello hello fox
...

we want to generate the following "inverted index"

fox →  (Document1: 1, 4)(Document2: 1, 3)(Document3: 4)
jumped  → (Document1: 2)(Document2: 4)
fast → (Document1: 3, 5)(Document2: 5)
ran  → (Document2: 2)
hello  → (Document3: 1, 2, 3)
...

The goal is to develop a classic MapReduce program 
for inverted index creation: generate a list of 
locations (word number in the document and identifier 
for the document) for each word occurrence. An 
identifier for each document is provided as the key 
to the map() function and value is a string of words 
(for example, "hello hello hello fox" will be the value 
for the "Document3").

a. Write a map() function 
you must identify Key and Value for the map()

b. Write a reduce() function (NOT a PySpark function): 
you must identify Key and Value for the reduce()

In [297]:
input_path='/Users/supriyatiwari/documents/PySpark_docs/inverted.txt'

In [298]:
rdd = spark.sparkContext.textFile(input_path)
rdd.collect()


['Document1: fox jumped fast fox fast',
 'Document2: fox ran fox jumped fast',
 'Document3: hello hello hello fox']

In [299]:
rdd1 = rdd.map(lambda a: a.split(":"))
rdd1.collect()

[['Document1', ' fox jumped fast fox fast'],
 ['Document2', ' fox ran fox jumped fast'],
 ['Document3', ' hello hello hello fox']]

In [300]:
def find_labels(text):
    l = []
    words = []
    words = text[1].split(" ")
    count = 1
    for word in words:
        l.append((word, (text[0], count)))
        count += 1
    return (l)

In [301]:
rdd2 = rdd1.map(lambda a: find_labels(a))
rdd2.collect()

[[('', ('Document1', 1)),
  ('fox', ('Document1', 2)),
  ('jumped', ('Document1', 3)),
  ('fast', ('Document1', 4)),
  ('fox', ('Document1', 5)),
  ('fast', ('Document1', 6))],
 [('', ('Document2', 1)),
  ('fox', ('Document2', 2)),
  ('ran', ('Document2', 3)),
  ('fox', ('Document2', 4)),
  ('jumped', ('Document2', 5)),
  ('fast', ('Document2', 6))],
 [('', ('Document3', 1)),
  ('hello', ('Document3', 2)),
  ('hello', ('Document3', 3)),
  ('hello', ('Document3', 4)),
  ('fox', ('Document3', 5))]]

In [302]:
rdd3 = rdd2.flatMap(lambda x: x).groupByKey().mapValues(lambda iter: list(iter))
rdd3.collect()

[('', [('Document1', 1), ('Document2', 1), ('Document3', 1)]),
 ('ran', [('Document2', 3)]),
 ('fox',
  [('Document1', 2),
   ('Document1', 5),
   ('Document2', 2),
   ('Document2', 4),
   ('Document3', 5)]),
 ('jumped', [('Document1', 3), ('Document2', 5)]),
 ('fast', [('Document1', 4), ('Document1', 6), ('Document2', 6)]),
 ('hello', [('Document3', 2), ('Document3', 3), ('Document3', 4)])]

In [303]:

def map_labels(i):
    d = {}
    for item in i:
        if item[0] in d:
            d[item[0]].append(item[1])
        else:
            d[item[0]] = [item[1]]
    return (d)      



In [304]:
invert = rdd3.mapValues(lambda i: map_labels(i)).sortByKey()
invert.collect()

[('', {'Document1': [1], 'Document2': [1], 'Document3': [1]}),
 ('fast', {'Document1': [4, 6], 'Document2': [6]}),
 ('fox', {'Document1': [2, 5], 'Document2': [2, 4], 'Document3': [5]}),
 ('hello', {'Document3': [2, 3, 4]}),
 ('jumped', {'Document1': [3], 'Document2': [5]}),
 ('ran', {'Document2': [3]})]

==========================================================================================

# 20. Assume that all of the input is in a file 
called  "/dir/movies.txt" and each input record 
has the following format:

<userID, movieID, rating-in-range-of-1-to-5>

Sample input:

user1,movie1,3
user1,movie1,1
user1,movie2,5
user2,movie1,4
...
...
... 

Note that a user may rate the same movie any 
number of times. The goal is to find the number 
of raters per movie.  Write a PySpark program 
(as a set of trasformations and actions) to 
accomplish this task. Your output will be

"movieID number-of-raters"

In [313]:
input_path='/Users/supriyatiwari/documents/PySpark_docs/movies.txt'

In [314]:
rdd21 = spark.sparkContext.textFile(input_path)


In [337]:
rdd21.collect()

['user1,movie1,3',
 'user1,movie1,1',
 'user1,movie2,5',
 'user2,movie1,4',
 'user1,movie1,3',
 'user1,movie1,1',
 'user1,movie2,5',
 'user2,movie1,4']

In [350]:
rdd22 = rdd21.map(lambda a: a.split(","))
rdd22.collect()

[['user1', 'movie1', '3'],
 ['user1', 'movie1', '1'],
 ['user1', 'movie2', '5'],
 ['user2', 'movie1', '4'],
 ['user1', 'movie1', '3'],
 ['user1', 'movie1', '1'],
 ['user1', 'movie2', '5'],
 ['user2', 'movie1', '4']]

In [351]:
rdd23 = rdd22.map(lambda a: (a[1],1))

In [352]:
rdd23.collect()

[('movie1', 1),
 ('movie1', 1),
 ('movie2', 1),
 ('movie1', 1),
 ('movie1', 1),
 ('movie1', 1),
 ('movie2', 1),
 ('movie1', 1)]

In [353]:
rdd24=rdd23.reduceByKey(lambda x,y:(x+y) )

In [354]:
rdd24.collect()

[('movie1', 6), ('movie2', 2)]

==========================================================================================

# 21. Consider the following in PySpark:

/data = [1, 1, 2, 3, 1, 2, 3, 3, 3]
/rdd = sc.parallelize(data, 3)
/rdd2 = rdd.map(lambda x: (x, 2))
 /groupedby = rdd2.groupByKey().collect()
 /reducedby = rdd2.reduceByKey(lambda x, y: x * y).collect()

a. Show the content of "groupedby" in detail
show your work...

b. Show the content of "reducedby" in detail
show your work...


In [356]:
data2 = [1, 1, 2, 3, 1, 2, 3, 3, 3]

In [357]:
rdd25 = sc.parallelize(data2, 3)

In [358]:
rdd25.collect()

[1, 1, 2, 3, 1, 2, 3, 3, 3]

In [359]:
rdd26 = rdd25.map(lambda x: (x, 2))

In [360]:
rdd26.collect()

[(1, 2), (1, 2), (2, 2), (3, 2), (1, 2), (2, 2), (3, 2), (3, 2), (3, 2)]

In [362]:
groupedby = rdd26.groupByKey()

==========================================================================================

# 23. Let genes.txt be a huge text file, where every 
record has the following format (reference can be 
in {1, 2, 3} where 1 denotes cancer, 2 denotes 
healthy, and 3 is undefined):

<geneID><,><reference><,><geneValue>

For example, a sample input might be:

g1,1, 2.3
g1,2, 1.5
g1,3,2.5
g1,1,4.1
g2,1,1.3
g2,2,1.8
g2,3,3.5
g2,1,4.3
g2,1,2.9
...
...
...

a. Write a PySpark command/tranformation to
convert genes.txt file into an RDD<String> 
and then output the number of elements in 
that RDD

b. Let rdd1 represents the genes.txt file in 
Spark.  Use rdd1 and write a PySpark command/
tranformation to generate the following output 
per geneID

<geneID> <C> <S>

where C is the number of cancer genes (for geneID) 
and S is the sum of values for the cancer gene.

c. Let rdd1 represents the genes.txt file in Spark.
Use rdd1 and write a PySpark filter to remove all 
undefined genes.


In [409]:
input_path= '/Users/supriyatiwari/documents/PySpark_docs/genes.txt'

In [410]:
rdd27 = spark.sparkContext.textFile(input_path)

In [411]:
rdd27.collect()

['g1,1,2.3',
 'g1,2,1.5',
 'g1,3,2.5',
 'g1,1,4.1',
 'g2,1,1.3',
 'g2,2,1.8',
 'g2,3,3.5',
 'g2,1,4.3',
 'g2,1,2.9']

In [412]:
rdd28=rdd27.map(lambda x: x.split(","))

In [413]:
rdd28.collect()

[['g1', '1', '2.3'],
 ['g1', '2', '1.5'],
 ['g1', '3', '2.5'],
 ['g1', '1', '4.1'],
 ['g2', '1', '1.3'],
 ['g2', '2', '1.8'],
 ['g2', '3', '3.5'],
 ['g2', '1', '4.3'],
 ['g2', '1', '2.9']]

In [426]:
rdd29=rdd28.map(lambda x: (x[0], float(x[2])))

In [427]:
rdd29.collect()

[('g1', 2.3),
 ('g1', 1.5),
 ('g1', 2.5),
 ('g1', 4.1),
 ('g2', 1.3),
 ('g2', 1.8),
 ('g2', 3.5),
 ('g2', 4.3),
 ('g2', 2.9)]

In [428]:
rdd30=rdd29.groupByKey()

In [429]:
rdd30.collect()

[('g1', <pyspark.resultiterable.ResultIterable at 0x7fad6d06db50>),
 ('g2', <pyspark.resultiterable.ResultIterable at 0x7fad6d06d550>)]

In [391]:
rdd30=rdd29.groupByKey()

In [393]:
rdd30.collect()

[('g1', <pyspark.resultiterable.ResultIterable at 0x7fad6d08edd0>),
 ('g2', <pyspark.resultiterable.ResultIterable at 0x7fad6d08ec90>)]

In [433]:
rdd31=rdd30.mapValues(lambda x: ( len(x), sum(x)))

In [434]:
rdd31.collect()

[('g1', (4, 10.399999999999999)), ('g2', (5, 13.799999999999999))]

In [438]:
rdd32=rdd27.filter(lambda x: x[0] !="")

In [439]:
rdd32.collect()

['g1,1,2.3',
 'g1,2,1.5',
 'g1,3,2.5',
 'g1,1,4.1',
 'g2,1,1.3',
 'g2,2,1.8',
 'g2,3,3.5',
 'g2,1,4.3',
 'g2,1,2.9']

==========================================================================================

# 26. Assume we have 100 billion numbers saved in a 
file called big.txt (one number per record) and 
the goal is to find the number of zeros, positives, 
and negatives for all of these numbers. Write a 
spark/pyspark program to accomplish this task. Your 
client has asked you to write an efficient program 
for this otherwise he will not pay any money for 
your software!



In [487]:
input_path= '/Users/supriyatiwari/documents/PySpark_docs/big.txt'

In [488]:
rdd33 = spark.sparkContext.textFile(input_path)

In [489]:
rdd33.collect()

['2', '4', '6', '-2', '0', '90', '20', '30', '0', '0', '-9']

In [462]:
#rdd34=rdd33.map(lambda x: (int(x)))

In [490]:
rdd34=rdd33.map(lambda x: x.split(" "))

In [491]:
rdd34.collect()

[['2'],
 ['4'],
 ['6'],
 ['-2'],
 ['0'],
 ['90'],
 ['20'],
 ['30'],
 ['0'],
 ['0'],
 ['-9']]

In [494]:
rdd35=rdd34.map(lambda x: int(x[0]))

In [495]:
rdd35.collect()

[2, 4, 6, -2, 0, 90, 20, 30, 0, 0, -9]

In [501]:
def find_nums(Partitions):
    ze=0
    pos=0
    nega=0
    for val in Partitions: 
        if (val == 0):
            ze+=1
        elif (val > 0):
            pos+=1
        else:
            nega+=1
    return [(ze, pos, nega)]

In [502]:
rdd36=rdd35.mapPartitions(find_nums)

In [503]:
rdd36.collect()

[(1, 4, 1), (2, 2, 1)]

In [504]:
rdd37=rdd36.reduce(lambda x,y: ((x[0]+y[0]), (x[1]+y[1]), (x[2]+y[2]) ))

In [505]:
rdd37

(3, 6, 2)

==========================================================================================

# 27. Assume that all of the input is in a file 
called  "movies.txt" and each input record has the 
following format:

<userID,movieID,rating-in-range-of-1-to-5>

Sample input:

user1,movie1,3
user1,movie1,1
user1,movie2,5
user2,movie1,4
...
...
...

Note that a user may rate the same movie any 
number of times.

a. The goal is to find the number of raters 
per movie.  Write a complete PySpark program 
(as a set of trasformations and actions) to 
accomplish this task. Your output will be

(movieID number-of-raters)

b. The goal is to find the number of unique 
movies rated by each user. Write a complete 
PySpark program (as a set of trasformations 
and actions) to accomplish this task. Your 
output will be

(userID number-of-unique-movies)

In [507]:
input_path='/Users/supriyatiwari/documents/PySpark_docs/movies.txt'

In [508]:
rdd38 = spark.sparkContext.textFile(input_path)

In [509]:
rdd38.collect()

['user1,movie1,3',
 'user1,movie1,1',
 'user1,movie2,5',
 'user2,movie1,4',
 'user1,movie1,3',
 'user1,movie1,1',
 'user1,movie2,5',
 'user2,movie1,4']

In [510]:
rdd39=rdd38.map(lambda x: x.split(","))

In [511]:
rdd39.collect()

[['user1', 'movie1', '3'],
 ['user1', 'movie1', '1'],
 ['user1', 'movie2', '5'],
 ['user2', 'movie1', '4'],
 ['user1', 'movie1', '3'],
 ['user1', 'movie1', '1'],
 ['user1', 'movie2', '5'],
 ['user2', 'movie1', '4']]

In [516]:
rdd40=rdd39.map(lambda x: ((x[1]), 1))

In [517]:
rdd40.collect()

[('movie1', 1),
 ('movie1', 1),
 ('movie2', 1),
 ('movie1', 1),
 ('movie1', 1),
 ('movie1', 1),
 ('movie2', 1),
 ('movie1', 1)]

In [520]:
rdd41=rdd40.reduceByKey(lambda x,y: (x+y))

In [521]:
rdd41.collect()

[('movie1', 6), ('movie2', 2)]

================================

In [528]:
rdd42=rdd39.map(lambda x: ((x[0]), (x[1])))

In [529]:
rdd42.collect()

[('user1', 'movie1'),
 ('user1', 'movie1'),
 ('user1', 'movie2'),
 ('user2', 'movie1'),
 ('user1', 'movie1'),
 ('user1', 'movie1'),
 ('user1', 'movie2'),
 ('user2', 'movie1')]

In [530]:
rdd43=rdd42.groupByKey()

In [531]:
rdd43.collect()

[('user1', <pyspark.resultiterable.ResultIterable at 0x7fad6e322d10>),
 ('user2', <pyspark.resultiterable.ResultIterable at 0x7fad6e322e10>)]

In [542]:
def uniq(values):
    new=[]
    for val in values:
        if val not in new:
            new.append(val)
    return new

In [543]:
rdd44=rdd43.mapValues(uniq)

In [544]:
rdd44.collect()

[('user1', ['movie1', 'movie2']), ('user2', ['movie1'])]

In [545]:
rdd45=rdd44.mapValues(lambda x: len(x))

In [546]:
rdd45.collect()

[('user1', 2), ('user2', 1)]

====================================================================================================================

# 28. Consider the following in PySpark:



>>> data = [1, 1, 1, 2, 3, 1, 2, 3, 3, 3]
>>> rdd = sc.parallelize(data, 3)
>>> rdd2 = rdd.map(lambda x: (x, x))
>>> groupedRDD = rdd2.groupByKey().mapValues(lambda x : sum(x)).collect()

Show the content of groupedRDD in detail. show your work...


>> (1,1),(1,1),(1,1).(2,2),(3,3),(1,1).(2,2),(3,3),(3,3),(3,3)

In [ ]:
<<<(1,[1,1,1,1]), (2,[2,2]),(3,[3,3,3,3])

In [ ]:
<<<(1,4), (2,2),(3,4)

====================================================================================================================

# 29. Consider the following in PySpark:


In [ ]:
>>> data = [1, 1, 1, 1, 2, 2, 3, 1, 2, 3, 3, 3]
>>> rdd = sc.parallelize(data)
>>> rdd2 = rdd.map(lambda x: (x+1, x))
>>> reducedRDD = rdd2.reduceByKey(lambda x, y: x + y).collect()

Show the content of reducedRDD in detail. show your work...

In [ ]:
<<<(2,1),(2,1),(2,1),(2,1),(3,2),(3,2),(4,3),(2,1),(3,2),(4,3),(4,3),(4,3)

In [ ]:
<<<(2,5), (3,6),(4, 12)

====================================================================================================================

# 30. Consider the following in PySpark:

In [ ]:


>>> data = [1, -1, 1, 1, 0, 0, 1, -2, 
            2, 3, 1, 2, -3, ...]
>>> rdd = sc.parallelize(data)

Write a series of spark transformations to split 
rdd into two RDDs: rddP will hold only non-negative 
numbers and rddN will hold only negative numbers


====================================================================================================================

# 31. Let genes.txt be a huge text file, where every 

In [ ]:

record has the following format (reference can be 
in {1, 2, 3}  where 1 denotes cancer, 2 denotes 
healthy, and 3 is undefined):

<geneID><,><reference><,><geneValue>

For example, a sample input might be:

g1,1,2.3
g1,2,1.5
g1,3,2.5
g1,1,4.1
g2,1,1.3
g2,2,1.8
g2,3,3.5
g2,1,4.3
g2,1,2.9
...
...
...

a. Write a PySpark command/tranformation to
convert genes.txt file into an RDD<String> and 
then output the number of elements in that RDD 
(the final result will be in rdd1)

b. Let rdd1 represents the genes.txt file 
in Spark ((as RDD<String>).  Use rdd1 and write 
a PySpark command/tranformation to generate the 
following output per geneID

<geneID> <M> <N>

where M is the number of cancer genes (for geneID) 
and N is the sum of values for the cancer genes.

c. Find sum of the values for all genes.

d.  Let rdd1 represents the genes.txt file 
(as RDD<String>) in Spark.  Use rdd1 and write 
a PySpark filter to keep only healthy genes.


DONE

====================================================================================================================

# 33. PySpark and MapReduce solutions:

In [ ]:
Given the following input 
(millions of (key, value) pairs), find average rating 
per movie [note that ratings of less than 2 must be 
ignored]. The same movie can be rated any number of times. 

Input is given as: Key is a movie-id and Value is 
a rating  between 1 and 5.

Key  Value
m1    1
m1    3
m1    1
m1    5
...  ...
m2    5
m2    4
...  ...

a. Write a map() function: must identify 
Key and Value and their data types for the map()

b. Show output of all mappers for movies { m1, m2 }

c. Write a reduce() function: must identify Key 
and Value for the reduce()

d. Show all input to all reducers for movies { m1, m2 }

e. Show output of all reducers for movies { m1, m2 }

f. Find average per movie by using reduceByKey()

g. Find average per movie by using groupByKey()

In [605]:
movi=[('m1',1),('m1',3), ('m1',1),('m1',4),('m2',5),('m2',4), ('m3',5), ('m4',3),('m4',4)]

In [606]:
rdd46=spark.sparkContext.parallelize(movi)

In [607]:
rdd46.collect()

[('m1', 1),
 ('m1', 3),
 ('m1', 1),
 ('m1', 4),
 ('m2', 5),
 ('m2', 4),
 ('m3', 5),
 ('m4', 3),
 ('m4', 4)]

In [608]:
rdd47=rdd46.mapValues(lambda x: (x, 1))

In [609]:
rdd47.collect()

[('m1', (1, 1)),
 ('m1', (3, 1)),
 ('m1', (1, 1)),
 ('m1', (4, 1)),
 ('m2', (5, 1)),
 ('m2', (4, 1)),
 ('m3', (5, 1)),
 ('m4', (3, 1)),
 ('m4', (4, 1))]

In [610]:
rdd48=rdd47.reduceByKey(lambda x,y: (x[0]+y[0] , x[1] + y[1]))

In [611]:
rdd48.collect()

[('m3', (5, 1)), ('m4', (7, 2)), ('m2', (9, 2)), ('m1', (9, 4))]

In [616]:
rdd49=rdd48.mapValues(lambda x: (x[0]/x[1]))

In [617]:
rdd49.collect()

[('m3', 5.0), ('m4', 3.5), ('m2', 4.5), ('m1', 2.25)]

Xxxxxxxxxxxx

In [618]:
rdd50=rdd46.groupByKey()

In [619]:
rdd51=rdd50.mapValues(lambda x: (sum(x)/len(x)))

In [620]:
rdd51.collect()

[('m3', 5.0), ('m4', 3.5), ('m2', 4.5), ('m1', 2.25)]

====================================================================================================================

# 34. Using MapReduce and PySpark, 
write a series of transformations and actions to 
eliminate all duplicate records from a given big 
file called bigfile.txt.  Your output will be all 
of unique records contained in bigfile.txt.

In [916]:
input_path= "/Users/supriyatiwari/documents/PySpark_docs/big.txt"

In [917]:
rdd95=spark.sparkContext.textFile(input_path)

In [918]:
rdd96=rdd95.map(lambda x: int(x))

In [919]:
rdd96.collect()

[2, 4, 6, -2, 0, 90, 20, 30, 0, 0, -9]

In [923]:
rdd97=rdd96.distinct()

In [924]:
rdd97.collect()

[2, 4, 6, -2, 0, 90, 20, 30, -9]

# 35. Assume the following input

In [ ]:

<Employee-ID,type>

where type can be:
"fulltime"
"parttime"
"contractor"

The goal is to write a PySpark program to count  
"fulltime" and "parttime" employees. Your output 
should be something like:

fulltime: <number-of-fulltime-employees>
parttime: <number-of-parttime-employees>



In [632]:
type=(12, "fulltime"), (13, "parttime" ),(14,"contractor"), (10, "parttime"),(1, "fulltime"),(18, "fulltime")

In [633]:
rdd52=spark.sparkContext.parallelize(type)

In [664]:
rdd53=rdd52.map(lambda x: (x[1], (x[0], int(1))))

In [665]:
rdd53.collect()

[('fulltime', (12, 1)),
 ('parttime', (13, 1)),
 ('contractor', (14, 1)),
 ('parttime', (10, 1)),
 ('fulltime', (1, 1)),
 ('fulltime', (18, 1))]

In [666]:
rdd54=rdd53.filter(lambda x: (x[0] != "contractor"))

In [667]:
rdd54.collect()

[('fulltime', (12, 1)),
 ('parttime', (13, 1)),
 ('parttime', (10, 1)),
 ('fulltime', (1, 1)),
 ('fulltime', (18, 1))]

In [674]:
rdd55=rdd54.groupByKey()

In [675]:
rdd56=rdd55.mapValues(lambda x: len(x))

In [676]:
rdd56.collect()

[('parttime', 2), ('fulltime', 3)]

====================================================================================================================

# 36. Given the following rdd in pyspark:

In [ ]:
>>> data = ['k1', 'k2', 'k1', 'k2', 
            'k1', 'k2', 'k3', 'k2', 'k4']
>>> rdd = sc.parallelize(data)

write a sequence of pyspark transformations 
and actions to find frequencies of all keys 
in data. Keep only the (key, frequency) 
pairs if the frequency is greater than one.

For this example, your solution should generate/output:
('k1', 3)
('k2', 4)


In [677]:
ke=['k1', 'k2', 'k1', 'k2', 'k1', 'k2', 'k3', 'k2', 'k4']

In [678]:
rdd57=spark.sparkContext.parallelize(ke)

In [679]:
rdd58=rdd57.map(lambda x: (x, 1))

In [680]:
rdd58.collect()

[('k1', 1),
 ('k2', 1),
 ('k1', 1),
 ('k2', 1),
 ('k1', 1),
 ('k2', 1),
 ('k3', 1),
 ('k2', 1),
 ('k4', 1)]

In [682]:
rdd59= rdd58.reduceByKey(lambda x,y: (x+y))

In [683]:
rdd59.collect()

[('k1', 3), ('k2', 4), ('k4', 1), ('k3', 1)]

In [684]:
rdd60=rdd59.filter(lambda x: (x[1]>1))

In [685]:
rdd60.collect()

[('k1', 3), ('k2', 4)]

====================================================================================================================

# 37. Consider the following in PySpark:

In [ ]:
>>> data = [1, 1, 1, 1, 2, 2, 3, 1, 2, 3, 3, 3]
>>> rdd = sc.parallelize(data)
>>> rdd2 = rdd.map(lambda x: (x+1, x-1))
>>> myoutput = rdd2.reduceByKey(lambda x, y: x + y).collect()

Show the content of myoutput in detail. show your work...

In [ ]:
<<<(2,0),(2,0),(2,0),(2,0),(3,1),(3,1),(2,0),(3,1),(4,2),(4,2),(4,2)

In [ ]:
<<<(2,0),(3,3),(4,6)

====================================================================================================================

# 40. Assume we have 100 billion numbers saved
in a file called big.txt (one number per record) 
and the goal is to find the number of positives 
(numbers greater than zero) and negatives (numbers 
less than zero) for all of these numbers. Write a 
Spark/PySpark program to accomplish this task. Your 
client has asked you to write an efficient program 
for this otherwise he will not pay any money for 
your software!

Answer is same as question 26!!!!!!!

====================================================================================================================

# 42. Assume that all of the input is in 

In [ ]:

a file called  "movies.txt" and each 
input record has the following format:

<userID><,><movieID><,><rating-in-range-of-1-to-5>

Sample input:

user1,movie1,3
user1,movie1,1
user1,movie2,5
user2,movie1,4
...
...
...


Note that a user may rate the same movie any 
number of times. You may use the following 
functions in your transformations:

getUser("userX,movieY,ratingN")   returns "userX"
getMovie("userX,movieY,ratingN")  returns "movieY"
getRating("userX,movieY,ratingN") returns ratingN

MUST use the provided functions.

a. The goal is to find the number of raters 
per movie.  Write a complete PySpark program 
(as a set of trasformations and actions) to 
accomplish this task. Your output will be

<movieID> <number-of-raters>

b. The goal is to find the number of unique 
movies rated by each user. Write a complete 
PySpark program (as a set of trasformations 
and actions) to accomplish this task. 
Your output will be

<userID> <number-of-unique-movies>


In [688]:
input_path= "/Users/supriyatiwari/documents/PySpark_docs/movies.txt"

In [689]:
rdd61=spark.sparkContext.textFile(input_path)

In [690]:
rdd61.collect()

['user1,movie1,3',
 'user1,movie1,1',
 'user1,movie2,5',
 'user2,movie1,4',
 'user1,movie1,3',
 'user1,movie1,1',
 'user1,movie2,5',
 'user2,movie1,4']

In [691]:
rdd62=rdd61.map(lambda x: x.split(","))

In [692]:
rdd62.collect()

[['user1', 'movie1', '3'],
 ['user1', 'movie1', '1'],
 ['user1', 'movie2', '5'],
 ['user2', 'movie1', '4'],
 ['user1', 'movie1', '3'],
 ['user1', 'movie1', '1'],
 ['user1', 'movie2', '5'],
 ['user2', 'movie1', '4']]

In [693]:
rdd63=rdd62.map(lambda x: (x[1], 1))

In [694]:
rdd63.collect()

[('movie1', 1),
 ('movie1', 1),
 ('movie2', 1),
 ('movie1', 1),
 ('movie1', 1),
 ('movie1', 1),
 ('movie2', 1),
 ('movie1', 1)]

In [695]:
rdd64=rdd63.reduceByKey(lambda x,y: (x+y))

In [696]:
rdd64.collect()

[('movie1', 6), ('movie2', 2)]

=============================================

In [702]:
rdd65=rdd61.map(lambda x: x.split(","))

In [703]:
rdd66=rdd65.map(lambda x: (x[0], x[1]))

In [704]:
rdd66.collect()

[('user1', 'movie1'),
 ('user1', 'movie1'),
 ('user1', 'movie2'),
 ('user2', 'movie1'),
 ('user1', 'movie1'),
 ('user1', 'movie1'),
 ('user1', 'movie2'),
 ('user2', 'movie1')]

In [705]:
rdd67=rdd66.groupByKey()

In [706]:
rdd67.collect()

[('user1', <pyspark.resultiterable.ResultIterable at 0x7fad6e851150>),
 ('user2', <pyspark.resultiterable.ResultIterable at 0x7fad6e851110>)]

In [707]:
def unique_movies(movies):
    new=[]
    for val in movies:
        if val not in new:
            new.append(val)
    return new

In [708]:
rdd68=rdd67.mapValues(unique_movies)

In [709]:
rdd68.collect()

[('user1', ['movie1', 'movie2']), ('user2', ['movie1'])]

# 43. Consider the following in PySpark:

In [ ]:
>>> data = [1, 1, 1, 2, 3, 1, 2, 3, 3, 3]
>>> rdd = sc.parallelize(data)
>>> rdd2 = rdd.map(lambda x: (x, x+2))
>>> groupedRDD = rdd2.groupByKey()
                 .mapValues(lambda x : sum(x))
                 .collect()

Show the content of groupedRDD in detail. show your work...

In [ ]:
<<<(1,3),(1,3),(1,3),(2,4),(3,5),(1,3),(2,4),(3,5),(3,5),(3,5)

In [ ]:
<<<(1,[3,3,3,3]), (2,[4,4]),(3,[5,5,5,5])

In [ ]:
<<<(1,12),(2,8),(3,20)

====================================================================================================================

# 44. Consider the following in PySpark:

In [ ]:
>>> data = [1, 1, 1, 1, 2, 2, 3, 1, 2, 3, 3, 3]
>>> rdd = sc.parallelize(data)
>>> rdd2 = rdd.map(lambda x: (x+1, x))
>>> reducedRDD = rdd2.reduceByKey(lambda x, y: x + y).collect()

Show the content of reducedRDD in detail. show your work...

====================================================================================================================

# 45. Consider the following in PySpark:

In [ ]:
>>> data = [1, -1, 1, 1, 0, 0, 1, -2, 2, 3, 1, 2, -3, ...]
>>> rdd = sc.parallelize(data)

Write a series of spark transformations to split rdd into 
two RDDs: rddP will hold only non-negative numbers
and rddN will hold only negative numbers


In [927]:
data45 = [1, -1, 1, 1, 0, 0, 1, -2, 2, 3, 1, 2, -3 ]

In [928]:
rdd110=spark.sparkContext.parallelize(data45)

In [929]:
rdd111=rdd110.filter(lambda x: x >0)

In [930]:
rdd112=rdd110.filter(lambda x: x <0)

In [931]:
rdd111.collect()

[1, 1, 1, 1, 2, 3, 1, 2]

In [932]:
rdd112.collect()

[-1, -2, -3]

# 46. Assume we have about 100 billion  
numbers saved in a file called big.txt 
(one number  per record) and the goal is 
to perform the following
in order (MUST USE PySpark):

a. create an RDD<Integer> as rdd
b. count the exact number of numbers in rdd
c. remove all negative numbers
d. count all remaining numbers

In [710]:
input_path='/Users/supriyatiwari/documents/PySpark_docs/big.txt'

In [711]:
rdd69=spark.sparkContext.textFile(input_path)

In [712]:
rdd69.collect()

['2', '4', '6', '-2', '0', '90', '20', '30', '0', '0', '-9']

In [713]:
rdd70=rdd69.map(lambda x: (int(x)))

In [714]:
rdd70.collect()

[2, 4, 6, -2, 0, 90, 20, 30, 0, 0, -9]

In [717]:
rdd70.count()

11

In [718]:
rdd71=rdd70.filter(lambda x: (x>=0))

In [720]:
rdd71.count()

9

==============================================================

# 48. Consider the following in PySpark:

In [ ]:
>>> data = [0, 1, 0, 1, -1, 1, 0, 2, 3, 1, -2, -3, 3, 3]
>>> rdd = sc.parallelize(data)
>>> rdd2 = rdd.filter(lambda v: v > 0)
>>> rdd3 = rdd2.map(lambda x: (x, x+2))
>>> reducedRDD = rdd3.reduceByKey(lambda x, y: x + y).collect()

Show the content of reducedRDD in detail. 
show your work...


In [ ]:
<<< [1,1,1,2,3,1,3,3]

In [ ]:
<<<[(1,3), (1,3), (1,3), (2,4),(3,5), (1,3), (3,5), (3,5)]

In [ ]:
<<<[(1,12), (2,4), (3,15)]

==============================================================================================

# 49. Consider the following in PySpark:

In [ ]:
>>> data = [("a", 1), ("a", 1), ("a", 3), 
            ("b", 1), ("b", 1), ("b", 2), 
            ...]
>>> rdd = sc.parallelize(data)

Write a series of spark transformations to find 
the maximum value per key.

In [721]:
data49=[("a", 1), ("a", 1), ("a", 3),("b", 1), ("b", 1), ("b", 2)]

In [722]:
rdd72=spark.sparkContext.parallelize(data49)

In [723]:
rdd73=rdd72.reduceByKey(lambda x,y: (max(x,y)) )

In [724]:
rdd73.collect()

[('b', 2), ('a', 3)]

==============================================================================================

# 50. Consider the following in PySpark:

In [ ]:
Let data represent a set of records:

>>> data = ["abc", "abc", "xyz", "xyz", "xyz", ...]
>>> rdd = sc.parallelize(data)

Write a series of spark transformations to eliminate
all duplicate records. For this example, the output
will be : ["abc", "xyz", ...]. NOTE that you can NOT
use unique() and distinct() transformations.

In [725]:
data50 = ["abc", "abc", "xyz", "xyz", "xyz"]

In [726]:
rdd74=spark.sparkContext.parallelize(data50)

In [727]:
rdd75=rdd74.map(lambda x: (x, 1))

In [728]:
rdd75.collect()

[('abc', 1), ('abc', 1), ('xyz', 1), ('xyz', 1), ('xyz', 1)]

In [729]:
rdd76=rdd75.groupByKey()

In [730]:
rdd76.collect()

[('xyz', <pyspark.resultiterable.ResultIterable at 0x7fad6e886b50>),
 ('abc', <pyspark.resultiterable.ResultIterable at 0x7fad6e886b10>)]

In [731]:
rdd77=rdd76.map(lambda x: x[0])

In [732]:
rdd77.collect()

['xyz', 'abc']

==============================================================================================

# 53. Given the following input (filebig.txt)

In [ ]:
 , using Spark's mapPartitions() write 
an efficient transformation to find minimum and 
maximum of all given numbers. Note that every 
record (single line of input) may have thousands 
of numbers.

Input is given as:

10,4,50,40,30, ...
10,60,50,20, ...
20,20,30,40,50,2, ...
...


# 54. Given

In [ ]:


>>> def myfunc(n):
...     if n < 0:
...             return [n, -n, -n]
...     else:
...             return []
...
>>>
>>> data = [0, 1, 2, -3, -4]
>>> rdd = spark.sparkContext.parallelize(data)
>>> rdd.collect()
[0, 1, 2, -3, -4]
>>> rdd.count()
5
>>> rdd3 = rdd.flatMap(myfunc).flatMap(myfunc)
>>> rdd3.collect()

What is the output of this program?

In [ ]:
(-3,3,3,-4,4,4),
<<<(-3,3,3,-4,4,4)

==============================================================================================

# 55. classic MapReduce and PySpark:

In [ ]:

Given the following (key, value) pairs 
(as input to map()):

<key-as-string> <value-as-integer>

Write a complete map()  and reduce() functions to 
find the median per key. Only, Output medians, which 
are greater than 10. 

==============================================================================================

# 56. Use PySpark to answer this question

In [ ]:
Assume that all of the input is in a file called  
"movies.txt" (with millions of records) and each 
input record has the following format:

<MOVIE-ID><,><rating-in-range-of-1-to-5>

Sample input:

movie1,3
movie1,1
movie1,5
movie2,5
movie2,4
movie2,3
...

Note that a user may rate the same movie any number of 
times.  You HAVE to use the following Python functions 
in your transformations (NOTE, you MUST NOT use the 
split() function at all).

getMovie("movie,rating")  returns "movie" as String
getRating("movie,rating") returns rating as Integer

a. The goal is to find the number of raters 
per movie.  Write a complete PySpark program (as a set of 
PySpark trasformations and actions) to accomplish this task. 
Your output will be like:

<MOVIE-ID> <number-of-raters>

b. The goal is to find the average rating 
per movie. Write a complete PySpark program (as a set of 
trasformations and actions) to accomplish this task. Your 
output will be as:

<MOVIE-ID> <average-rating-per-MOVIE-ID>

In [749]:
input_path='/Users/supriyatiwari/documents/PySpark_docs/movie.txt'

In [750]:
rdd78=spark.sparkContext.textFile(input_path)

In [751]:
rdd78.collect()

['movie1,3', 'movie1,1', 'movie1,5', 'movie2,5', 'movie2,4', 'movie2,3']

In [752]:
rdd78.count()

6

In [753]:
def getMovie(values):
    for val in values: 
        

SyntaxError: invalid syntax (<ipython-input-753-a77666da0adb>, line 1)

==============================================================================================

# 57.  Consider the following in PySpark:

In [ ]:
>>> data = [0, 2, 2, -3, 1, -1, 3, -2, -4, 3]
>>> rdd = spark.sparkContext.parallelize(data)
>>> print("output-1: ", rdd.collect())
>>> rdd2 = rdd.filter(lambda v: v > 0)
>>> print("output-2: ", rdd2.collect())
>>> rdd3 = rdd2.map(lambda x: (x, x+2))
>>> print("output-3: ", rdd3.collect())
>>> rdd4 = rdd3.reduceByKey(lambda x, y: x + y)
>>> print("output-4: ", rdd4.collect())

Show the output in detail.  show your work...

In [754]:
data = [0, 2, 2, -3, 1, -1, 3, -2, -4, 3]

In [755]:
rdd = spark.sparkContext.parallelize(data)

In [756]:
print("output-1: ", rdd.collect())

output-1:  [0, 2, 2, -3, 1, -1, 3, -2, -4, 3]


In [757]:
rdd2 = rdd.filter(lambda v: v > 0)

In [758]:
print("output-2: ", rdd2.collect())

output-2:  [2, 2, 1, 3, 3]


In [759]:
rdd3 = rdd2.map(lambda x: (x, x+2))

In [760]:
print("output-3: ", rdd3.collect())

output-3:  [(2, 4), (2, 4), (1, 3), (3, 5), (3, 5)]


In [761]:
rdd4 = rdd3.reduceByKey(lambda x, y: x + y)

In [762]:
print("output-4: ", rdd4.collect())

output-4:  [(1, 3), (2, 8), (3, 10)]


==============================================================================================

# 61. Consider the following in PySpark:


In [ ]:
>>> def fun7(x):
>>>     if (x == 1):
>>>         return [x, 1]
>>>     if (x > 0):
>>>         return [x, x, -2]
>>>     return []
>>> #end-def
>>> data = [1, 1, -1, -2, 2, 2, -4]
>>> rdd = spark.sparkContext.parallelize(data, 3)
>>> rdd2 = rdd.flatMap(fun7)
>>> rdd2.collect()
>>> pairs = rdd2.map(lambda x: (x, 3))
                .groupByKey()
                .mapValues(lambda x : sum(x))
                .collect()

Show the output. show your work...


In [769]:
def fun7(x):
    if (x == 1):
        return [x, 1]
    if (x > 0):
        return [x, x, -2]
    return []

In [770]:
data = [1, 1, -1, -2, 2, 2, -4]

In [771]:
rdd = spark.sparkContext.parallelize(data, 3)

In [772]:
rdd2 = rdd.flatMap(fun7)

In [773]:
rdd2.collect()

[1, 1, 1, 1, 2, 2, -2, 2, 2, -2]

In [777]:
pairs = rdd2.map(lambda x: (x, 3)).groupByKey().mapValues(lambda x : sum(x))

In [778]:
pairs.collect()

[(1, 12), (-2, 6), (2, 12)]

==============================================================================================

# 62. Consider the following 

In [ ]:
(key, value) pairs in PySpark:

>>> data = [('A', 4), ('A', 8), 
            ('B', 5), ('B', 7), ...]
>>> rdd = sc.parallelize(data)

Write a set of Spark transformations to 
find the average (mean) value per key. 
You CAN NOT use groupByKey().
Show your work in detail. 

In [795]:
data62 = [('A', 4), ('A', 8), ('B', 5), ('B', 7)]

In [796]:
rdd79=spark.sparkContext.parallelize(data62)

In [801]:
rdd79.collect()

[('A', 4), ('A', 8), ('B', 5), ('B', 7)]

In [802]:
rdd80=rdd79.mapValues(lambda x: (x,1))

In [803]:
rdd80.collect()

[('A', (4, 1)), ('A', (8, 1)), ('B', (5, 1)), ('B', (7, 1))]

In [806]:
rdd81=rdd80.reduceByKey(lambda x,y: ((x[0] + y[0]), (x[1]+y[1])))

In [807]:
rdd81.collect()

[('A', (12, 2)), ('B', (12, 2))]

In [808]:
final = rdd81.mapValues(lambda x: (x[0]/x[1]))

In [809]:
final.collect()

[('A', 6.0), ('B', 6.0)]

==============================================================================================

# 64. Given the following rdd of pairs 
in PySpark:


In [ ]:
>>> data = [('k1', 5), ('k1', 6), ('k1', 7), 
            ('k2', 7), ('k2', 8), ...]
>>> rdd = spark.sparkContext.parallelize(data)

write a sequence of pyspark transformations and actions 
to find unique list of keys: {'k1', 'k2', ...}

In [810]:
data64=[('k1', 5), ('k1', 6), ('k1', 7), ('k2', 7), ('k2', 8)]

In [811]:
rdd82= spark.sparkContext.parallelize(data64)

In [813]:
rdd83=rdd82.groupByKey()

In [814]:
rdd83.collect()

[('k1', <pyspark.resultiterable.ResultIterable at 0x7fad6e914490>),
 ('k2', <pyspark.resultiterable.ResultIterable at 0x7fad6e914450>)]

In [815]:
rdd84=rdd83.map(lambda x: x[0])

In [816]:
rdd84.collect()

['k1', 'k2']

# 65. Given the following rdd of 


pairs in PySpark:

>>> data = [('k1', 5), ('k1', 6), ('k1', 7), 
            ('k2', 7), ('k2', 8), ...]
>>> rdd = spark.sparkContext.parallelize(data)

write a sequence of pyspark transformations to 
find the minimum value per key.


In [817]:
data65 = [('k1', 5), ('k1', 6), ('k1', 7), ('k2', 7), ('k2', 8)]

In [818]:
rdd85=spark.sparkContext.parallelize(data65)

In [819]:
rdd86= rdd85.reduceByKey(lambda x,y:(min(x,y)) )

In [820]:
rdd86.collect()

[('k1', 5), ('k2', 7)]

==============================================================================================

# 66. Given the following rdd of 

In [ ]:

pairs in PySpark:

>>> data = [('a', 2), ('b', 3), ('d', 2), 
            ('x', 3), ('y', 1), ...]
>>> rdd = spark.sparkContext.parallelize(data)

write a sequence of pyspark transformations to 
generate the following output: MUST use flatMap():

[
 'a', 'a', 
 'b', 'b', 'b', 
 'd', 'd', 
 'x', 'x', 'x', 
 'y',
  ...
]


In [1058]:
data66=[('a', 2), ('b', 3), ('d', 2), ('x', 3), ('y', 1)]

In [1059]:
rdd87 = spark.sparkContext.parallelize(data66)

In [1060]:
rdd88=rdd87.map(lambda x: (x[0] * x[1]))

In [1061]:
rdd88.collect()

['aa', 'bbb', 'dd', 'xxx', 'y']

In [1062]:
rddrep=rdd88.flatMap(lambda x: x)

In [1063]:
rddrep.collect()

['a', 'a', 'b', 'b', 'b', 'd', 'd', 'x', 'x', 'x', 'y']

# 67:  Consider the following RDD:

In [ ]:

>>> input = [("k1", "v1"), ("k1", "v1"), ("k1", "v2"), ...]
>>> RDD1 = sc.parallelize(input)
The goal is to write a set of Spark transformations to generate
unique (K, V) pairs [combination of K and V must be unique]. You
may NOT use Spark's distinct() function.

In [1052]:
input67 = [("k1", "v1"), ("k1", "v1"), ("k1", "v2"), ("k1", "v2"), ("k1", "v2"), ("k3", "v2") ]

In [1053]:
rdd89=spark.sparkContext.parallelize(input67)

In [1054]:
rdd90=rdd89.map(lambda x: (x,1))

In [1055]:
rdd91=rdd90.groupByKey()   

In [1056]:
rdd222=rdd91.map(lambda x: (x[0]))

In [1057]:
rdd222.collect()

[('k3', 'v2'), ('k1', 'v1'), ('k1', 'v2')]

# 73. Consider the following input:

In [ ]:

1
11
-1
2
12
3
-4
13
4
14
...

Suppose, we want to  count all positive odd and even numbers. 
Write an efficient PySpark program to accomplish this task.

In [905]:
input73=[1,11,-1,2,12,3,-4,13,4]

In [906]:
rdd92=spark.sparkContext.parallelize(input73)

In [907]:
rdd93= rdd92.filter(lambda x: (x>0))

In [908]:
rdd94=rdd93.reduce(lambda x,y: (x+y))

In [910]:
rdd94

46

# 2. Given RDD[(String, Integer)], find median per key.  
Provide two solutions: using groupByKey() and reduceByKey()

In [1091]:
import numpy as np

In [1092]:
data2000=[('Abc',3),('Abc',1),('Abc',2),('Abc',3),('Bdc',4),('Bdc',4),('Bdc',2),('Bdc',1),('Bdc',45)]

In [1093]:
rdd2000=spark.sparkContext.parallelize(data2000)

In [1094]:
pairs = rdd2000.map(lambda x: ((x[0]), (int(x[1]))))

In [1095]:
pairs.collect()

[('Abc', 3),
 ('Abc', 1),
 ('Abc', 2),
 ('Abc', 3),
 ('Bdc', 4),
 ('Bdc', 4),
 ('Bdc', 2),
 ('Bdc', 1),
 ('Bdc', 45)]

In [1096]:
def find_median(x):
    array_list = x[1]
    med = np.median(array_list)
    return(x[0], med)

In [1097]:
array_key = pairs.groupByKey().mapValues(lambda iter: list(iter)).sortByKey()
array_key.collect()

[('Abc', [3, 1, 2, 3]), ('Bdc', [4, 4, 2, 1, 45])]

In [1098]:
key_med = array_key.map(lambda rec: find_median(rec))

In [1099]:
key_med.collect()

[('Abc', 2.5), ('Bdc', 4.0)]

==============================================================================================

In [1107]:
data2000=[('Abc',3),('Abc',1),('Abc',2),('Abc',3),('Bdc',4),('Bdc',4),('Bdc',2),('Bdc',1),('Bdc',45)]

In [1108]:
rdd2000=spark.sparkContext.parallelize(data2000)

In [1114]:
from numpy import median

In [1120]:
def find_median(x):
    med = median(x)
    return(med)

In [1121]:
rdd2001=rdd2000.groupByKey().mapValues(lambda x: (list(x)))

In [1122]:
rdd2002=rdd2001.sortByKey()

In [1123]:
rdd2003=rdd2002.mapValues(find_median)

In [1124]:
rdd2003.collect()

[('Abc', 2.5), ('Bdc', 4.0)]